In [1]:
import os
import json
import requests
import pandas as pd
from mysql import connector
from dotenv import load_dotenv
import numpy as np

In [2]:
coins = ['bitcoin', 'ethereum']

In [3]:
load_dotenv()

True

In [4]:
url = "https://api.coingecko.com/api/v3/coins/markets"
params = {
    "vs_currency": "usd",
    "ids": ",".join(coins),
    "order": "market_cap_desc",
    "per_page": len(coins),
    "page": 1,
    "sparkline": "false"
}

In [5]:
print(params)

{'vs_currency': 'usd', 'ids': 'bitcoin,ethereum', 'order': 'market_cap_desc', 'per_page': 2, 'page': 1, 'sparkline': 'false'}


# EXTRACT

In [6]:
response = requests.get(url, params=params)

In [7]:
payload = response.json()

In [8]:
payload

[{'id': 'bitcoin',
  'symbol': 'btc',
  'name': 'Bitcoin',
  'image': 'https://coin-images.coingecko.com/coins/images/1/large/bitcoin.png?1696501400',
  'current_price': 115787,
  'market_cap': 2306499951848,
  'market_cap_rank': 1,
  'fully_diluted_valuation': 2306499951848,
  'total_volume': 19848268311,
  'high_24h': 116154,
  'low_24h': 115157,
  'price_change_24h': 443.96,
  'price_change_percentage_24h': 0.38491,
  'market_cap_change_24h': 8252725083,
  'market_cap_change_percentage_24h': 0.35909,
  'circulating_supply': 19923296.0,
  'total_supply': 19923296.0,
  'max_supply': 21000000.0,
  'ath': 124128,
  'ath_change_percentage': -6.74703,
  'ath_date': '2025-08-14T00:37:02.582Z',
  'atl': 67.81,
  'atl_change_percentage': 170604.59777,
  'atl_date': '2013-07-06T00:00:00.000Z',
  'roi': None,
  'last_updated': '2025-09-20T19:19:22.308Z'},
 {'id': 'ethereum',
  'symbol': 'eth',
  'name': 'Ethereum',
  'image': 'https://coin-images.coingecko.com/coins/images/279/large/ethereum.p

In [9]:
formatted_response = json.dumps(payload, indent=4)
print(formatted_response)

[
    {
        "id": "bitcoin",
        "symbol": "btc",
        "name": "Bitcoin",
        "image": "https://coin-images.coingecko.com/coins/images/1/large/bitcoin.png?1696501400",
        "current_price": 115787,
        "market_cap": 2306499951848,
        "market_cap_rank": 1,
        "fully_diluted_valuation": 2306499951848,
        "total_volume": 19848268311,
        "high_24h": 116154,
        "low_24h": 115157,
        "price_change_24h": 443.96,
        "price_change_percentage_24h": 0.38491,
        "market_cap_change_24h": 8252725083,
        "market_cap_change_percentage_24h": 0.35909,
        "circulating_supply": 19923296.0,
        "total_supply": 19923296.0,
        "max_supply": 21000000.0,
        "ath": 124128,
        "ath_change_percentage": -6.74703,
        "ath_date": "2025-08-14T00:37:02.582Z",
        "atl": 67.81,
        "atl_change_percentage": 170604.59777,
        "atl_date": "2013-07-06T00:00:00.000Z",
        "roi": null,
        "last_updated": "2025

# TRANSFORM

In [10]:
coins_df = pd.DataFrame(payload)
coins_df

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://coin-images.coingecko.com/coins/images...,115787.00,2306499951848,1,2306499951848,19848268311,116154.00,...,1.992330e+07,21000000.0,124128.00,-6.74703,2025-08-14T00:37:02.582Z,67.810000,1.706046e+05,2013-07-06T00:00:00.000Z,None,2025-09-20T19:19:22.308Z
1,ethereum,eth,Ethereum,https://coin-images.coingecko.com/coins/images...,4481.44,540518033296,2,540518033296,15620991634,4506.69,...,1.207039e+08,NaN,4946.05,-9.47941,2025-08-24T19:21:03.333Z,0.432979,1.033944e+06,2015-10-20T00:00:00.000Z,"{'times': 50.739659997802214, 'currency': 'btc...",2025-09-20T19:19:23.107Z


In [11]:
coins_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 26 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                2 non-null      object 
 1   symbol                            2 non-null      object 
 2   name                              2 non-null      object 
 3   image                             2 non-null      object 
 4   current_price                     2 non-null      float64
 5   market_cap                        2 non-null      int64  
 6   market_cap_rank                   2 non-null      int64  
 7   fully_diluted_valuation           2 non-null      int64  
 8   total_volume                      2 non-null      int64  
 9   high_24h                          2 non-null      float64
 10  low_24h                           2 non-null      float64
 11  price_change_24h                  2 non-null      float64
 12  price_change

In [12]:
coins_df = coins_df.drop(columns=['image', 'fully_diluted_valuation', 'price_change_24h', 'market_cap_change_24h', 'total_supply', 'roi'], errors='ignore')

coins_df.info()

coins_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 20 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                2 non-null      object 
 1   symbol                            2 non-null      object 
 2   name                              2 non-null      object 
 3   current_price                     2 non-null      float64
 4   market_cap                        2 non-null      int64  
 5   market_cap_rank                   2 non-null      int64  
 6   total_volume                      2 non-null      int64  
 7   high_24h                          2 non-null      float64
 8   low_24h                           2 non-null      float64
 9   price_change_percentage_24h       2 non-null      float64
 10  market_cap_change_percentage_24h  2 non-null      float64
 11  circulating_supply                2 non-null      float64
 12  max_supply  

,id,symbol,name,current_price,market_cap,market_cap_rank,total_volume,high_24h,low_24h,price_change_percentage_24h,market_cap_change_percentage_24h,circulating_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,last_updated
0,bitcoin,btc,Bitcoin,115787.00,2306499951848,1,19848268311,116154.00,115157.00,0.38491,0.35909,1.992330e+07,21000000.0,124128.00,-6.74703,2025-08-14T00:37:02.582Z,67.810000,1.706046e+05,2013-07-06T00:00:00.000Z,2025-09-20T19:19:22.308Z
1,ethereum,eth,Ethereum,4481.44,540518033296,2,15620991634,4506.69,4443.55,0.46267,0.38651,1.207039e+08,NaN,4946.05,-9.47941,2025-08-24T19:21:03.333Z,0.432979,1.033944e+06,2015-10-20T00:00:00.000Z,2025-09-20T19:19:23.107Z


In [13]:
coins_df['ath_date'] = pd.to_datetime(coins_df['ath_date'], errors='coerce')
coins_df['atl_date'] = pd.to_datetime(coins_df['atl_date'], errors='coerce')
coins_df['last_updated'] = pd.to_datetime(coins_df['last_updated'], errors='coerce')

decimals_dict = {
    'current_price': 7,
    'high_24h': 8,
    'low_24h': 8,
    'price_change_percentage_24h': 4,
    'market_cap_change_percentage_24h': 4,
    'circulating_supply': 8,
    'max_supply': 8,
    'ath': 8,
    'ath_change_percentage': 4,
    'atl': 8,
    'atl_change_percentage': 4
}

coins_df = coins_df.round(decimals_dict)

coins_df = coins_df.replace({np.nan: None})

coins_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 20 columns):
 #   Column                            Non-Null Count  Dtype              
---  ------                            --------------  -----              
 0   id                                2 non-null      object             
 1   symbol                            2 non-null      object             
 2   name                              2 non-null      object             
 3   current_price                     2 non-null      object             
 4   market_cap                        2 non-null      int64              
 5   market_cap_rank                   2 non-null      int64              
 6   total_volume                      2 non-null      int64              
 7   high_24h                          2 non-null      object             
 8   low_24h                           2 non-null      object             
 9   price_change_percentage_24h       2 non-null      object             

# LOAD

In [14]:
MYSQL_USER = os.getenv("MYSQL_USER")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")
MYSQL_HOST = os.getenv("MYSQL_HOST")
MYSQL_PORT = os.getenv("MYSQL_PORT")
MYSQL_DB = os.getenv("MYSQL_DATABASE")

In [15]:
db_conn = connector.connect(
    host=MYSQL_HOST,
    user=MYSQL_USER,
    password=MYSQL_PASSWORD,
    port=MYSQL_PORT,
    database=MYSQL_DB,
    connection_timeout=10,
    autocommit=False,
    raise_on_warnings=True
)   

db_cur = db_conn.cursor()
print(f"[SUCCESS] Connected to MySQL db {MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB} as user {MYSQL_USER}")


[SUCCESS] Connected to MySQL db localhost:3306/cryptodb as user root


In [16]:
sql_table = "market_snapshots"
db_cur.execute(f"SHOW TABLES LIKE '{sql_table}'")

if db_cur.fetchone() is None:
    raise SystemExit(f"[ERROR] Table '{sql_table}' does not exist in database '{MYSQL_DB}'")
else:
    print(f"[SUCCESS] Table '{sql_table}' exists in database '{MYSQL_DB}'")
    

[SUCCESS] Table 'market_snapshots' exists in database 'cryptodb'


In [17]:
INSERT_SQL = f"""
INSERT INTO {sql_table} (
    coin_id, symbol, name, current_price, market_cap, 
    market_cap_rank, total_volume, high_24h, low_24h, 
    price_change_percentage_24h, market_cap_change_percentage_24h, 
    circulating_supply, max_supply, ath, ath_change_percentage, 
    ath_date, atl, atl_change_percentage, atl_date, last_updated
) 
VALUES (
    %s, %s, %s, %s, %s, 
    %s, %s, %s, %s, 
    %s, %s, 
    %s, %s, %s, %s, 
    %s, %s, %s, %s, %s
) 
"""


In [18]:
coins_list = coins_df.values.tolist()
print(coins_list)

[['bitcoin', 'btc', 'Bitcoin', 115787.0, 2306499951848, 1, 19848268311, 116154.0, 115157.0, 0.3849, 0.3591, 19923296.0, 21000000.0, 124128.0, -6.747, Timestamp('2025-08-14 00:37:02.582000+0000', tz='UTC'), 67.81, 170604.5978, Timestamp('2013-07-06 00:00:00+0000', tz='UTC'), Timestamp('2025-09-20 19:19:22.308000+0000', tz='UTC')], ['ethereum', 'eth', 'Ethereum', 4481.44, 540518033296, 2, 15620991634, 4506.69, 4443.55, 0.4627, 0.3865, 120703883.6559501, None, 4946.05, -9.4794, Timestamp('2025-08-24 19:21:03.333000+0000', tz='UTC'), 0.432979, 1033944.0438, Timestamp('2015-10-20 00:00:00+0000', tz='UTC'), Timestamp('2025-09-20 19:19:23.107000+0000', tz='UTC')]]


In [19]:
try:
    db_cur.executemany(INSERT_SQL, coins_list)
    db_conn.commit()
    print(f"[SUCCESS] Inserted {db_cur.rowcount} records into table '{sql_table}'")
except connector.Error as err:
    db_conn.rollback()
    print(f"[ERROR] Failed to insert records into table '{sql_table}': {err}")
finally:
    db_cur.close()
    db_conn.close()
    print("[INFO] MySQL connection closed")

[SUCCESS] Inserted 2 records into table 'market_snapshots'
[INFO] MySQL connection closed
